<a href="https://colab.research.google.com/github/SinghPrach/BrushUp_Python/blob/main/GoogleColab_ATS_ResumeScanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://oindrilasen.com/2021/05/build-resume-scanner-using-python-nlp/

In [82]:
!pip install docx2txt

In [83]:
import io
import docx2txt
import re
import operator
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import spacy
nltk.download('punkt')
nltk.download('stopwords')
set(stopwords.words('english'))
from wordcloud import WordCloud
from collections import Counter
from nltk.probability import FreqDist
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Read Word Resume

In [84]:
def read_word(word_doc):
     resume = docx2txt.process(word_doc)
     resume = str(resume)
     #print(resume)
     text =  ''.join(resume)
     text = text.replace("\n", "")
     if text:
         return text

In [85]:
resume_w = read_word('/content/Resume/PrachiSingh.docx')
job_w = read_word('/content/Resume/jobdesc.docx')

In [86]:
# print(resume_w)

Using Cosine Similarity Approach

In [87]:
def cosineSimilar_get_resume_score(text):
    cv = CountVectorizer(stop_words='english')
    count_matrix = cv.fit_transform(text)
    #Print the similarity scores
    print("\nSimilarity Scores:")

    #get the match percentage
    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage, 2) # round to two decimal

    print("Using Cosine Similarity approach, the entered resume matches about "+ str(matchPercentage)+ "% of the job description.")

In [88]:
# For the same job description, same resume got two different scores, when entered in 2 different formats.
# 1. .pdf and 2. .docx
# In pdf format, the score was 6.82%
# In docx format, the score was 35.21%
#### Conclusion: While applying, always apply in the .docx format

# **GET SIMILARITY SCORE BETWEEN YOUR RESUME AND THE JOB DESCRIPTION**

In [89]:
text = [resume_w, job_w]
    ## Get a Match score
cosineSimilar_get_resume_score(text)


Similarity Scores:
Using Cosine Similarity approach, the entered resume matches about 65.4% of the job description.


# **Getting the keywords to improve Resume Score**

In [90]:
def clean_job_desc(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # remove trailing spaces
    text = text.strip()
    # remove numbers
    text = re.sub('[0-9]+', '', text)
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [91]:
cleaned_jd = clean_job_desc(job_w)

In [92]:
print(type(cleaned_jd))

<class 'str'>


In [93]:
# Generate the word cloud
# wordcloud = WordCloud(width=800, height=400, background_color='white').generate(cleaned_jd)

# # Display the word cloud
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')
# plt.show()


Recommend keywords using TF-IDF Approach

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=100)  # Adjust max_features as needed
X = vectorizer.fit_transform([cleaned_jd])

# Get feature names and their scores
features = vectorizer.get_feature_names_out()
scores = X.sum(axis=0).A1

# Pair features with their scores and sort
keyword_scores = dict(zip(features, scores))
sorted_keywords = sorted(keyword_scores.items(), key=lambda item: item[1], reverse=True)

# print("Keywords:", sorted_keywords)

df_keywords_tfdif = pd.DataFrame(sorted_keywords, columns=['Keyword', 'Percent'])

display(df_keywords_tfdif)

,Keyword,Percent
0,data,0.781962
1,azure,0.456145
2,pipelines,0.097745
3,services,0.097745
4,sql,0.097745
...,...,...
95,understand,0.032582
96,uptodate,0.032582
97,using,0.032582
98,visualizationcollaborate,0.032582


Recommend keywords using Named Entity Recognition (NER)

In [96]:
# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Process the text
doc = nlp(job_w)

# Extract named entities
entities = [(ent.text, ent.label_) for ent in doc.ents]

print("Named Entities:", entities)

df_entity = pd.DataFrame(entities, columns=['Keyword', 'Type'])

display(df_entity)

Named Entities: [('Azure Data Factory', 'ORG'), ('Azure Databricks', 'ORG'), ('Azure SQL Datawarehouse', 'ORG'), ('Bachelor', 'ORG'), ('Computer Science, Engineering', 'ORG'), ('Azure Data Factory', 'ORG'), ('Azure Databricks', 'ORG'), ('SQL Datawarehouse', 'PERSON'), ('Azure Data Lake Storage', 'ORG'), ('SQL', 'ORG'), ('Scala', 'DATE'), ('ETL', 'ORG'), ('ELT', 'ORG'), ('Engineer Associate', 'PERSON')]


,Keyword,Type
0,Azure Data Factory,ORG
1,Azure Databricks,ORG
2,Azure SQL Datawarehouse,ORG
3,Bachelor,ORG
4,"Computer Science, Engineering",ORG
5,Azure Data Factory,ORG
6,Azure Databricks,ORG
7,SQL Datawarehouse,PERSON
8,Azure Data Lake Storage,ORG
9,SQL,ORG
